In [4]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFT, PauliEvolutionGate
from qiskit_aer import Aer
import json
from qiskit.quantum_info import SparsePauliOp


def estimate_ground_energy_qpe(
    H: "SparsePauliOp",
    num_eval_qubits: int = 10,
    shots: int = 4096,
    t: float | None = None,
    psi0_circuit: QuantumCircuit | None = None,
    backend_name: str = "aer_simulator"
):
    """
    H: SparsePauliOp（既にロード済みの qubit_ham_loaded）
    num_eval_qubits: 位相推定の補助ビット数（10前後が初手の目安）
    shots: サンプリング回数
    t: 時間スケール。未指定なら ||H||_1（係数和）から自動設定して位相折り返しを避ける
    psi0_circuit: 初期状態回路（Noneなら |0...0>）
    """

    # --- 1) 位相折り返しを避ける t を自動設定 ---
    # e^{-i H t} の固有位相は E * t (mod 2π)
    # 最大固有値幅の上界として Σ|c_i| を使う（Pauliの固有値は±1）
    if t is None:
        coeff_sum = float(np.sum(np.abs(H.coeffs)))
        if coeff_sum == 0:
            raise ValueError("Hamiltonian has zero coefficients; cannot pick a meaningful t.")
        # 安全側に π/2 のマージンをとる
        t = np.pi / (2.0 * coeff_sum)

    n = H.num_qubits
    qr_anc = QuantumRegister(num_eval_qubits, "anc")
    qr_sys = QuantumRegister(n, "q")
    cr = ClassicalRegister(num_eval_qubits, "c")
    qc = QuantumCircuit(qr_anc, qr_sys, cr)

    # --- 2) 初期状態を用意（省略時は |0...0>） ---
    if psi0_circuit is not None:
        if psi0_circuit.num_qubits != n:
            raise ValueError("psi0_circuit has different number of qubits from the Hamiltonian.")
        qc.compose(psi0_circuit, qr_sys, inplace=True)

    # --- 3) Hadamard on ancillas ---
    qc.h(qr_anc)

    # --- 4) Controlled-U^{2^k} を適用（U = e^{-i H t}）---
    # PauliEvolutionGate は e^{-i H * time} を実装。これを制御化して ancilla_k を制御ビットに。
    base_evo = PauliEvolutionGate(H, time=t)
    for k in range(num_eval_qubits):
        evo_k = PauliEvolutionGate(H, time=t * (2 ** k)).control(1)
        qc.append(evo_k, [qr_anc[k], *qr_sys])

    # --- 5) 逆QFT on ancillas ---
    iqft = QFT(num_eval_qubits, do_swaps=True, approximation_degree=0).inverse()
    qc.append(iqft, qr_anc)

    # --- 6) 測定 ---
    qc.measure(qr_anc, cr)

    # --- 7) 実行 ---
    backend = Aer.get_backend(backend_name)
    tqc = transpile(qc, backend=backend)
    result = backend.run(tqc, shots=shots).result()
    counts = result.get_counts()

    # --- 8) 最頻ビット列 → 位相φ → エネルギーE へ変換 ---
    bitstring = max(counts, key=counts.get)  # 最頻値
    # Qiskitのカウントbit順は c[0] が最下位ビット（LSB）に来やすいので反転に注意
    # ここでは古典bit順が MSB→LSB と仮定して整数化（必要なら[::-1] を試してください）
    k = int(bitstring, 2)
    phi = k / (2 ** num_eval_qubits)  # φ in [0,1)
    # 位相角 θ = 2πφ。E = θ / t。ただし (-π, π] に折り返してエネルギー領域へマップ
    theta = 2.0 * np.pi * phi
    if theta > np.pi:
        theta -= 2.0 * np.pi
    energy_est = theta / t

    return {
        "energy": energy_est,
        "phase": phi,
        "bitstring": bitstring,
        "t": t,
        "num_eval_qubits": num_eval_qubits,
        "counts": counts,
        "circuit": qc
    }

# ===== 実行例 =====
# qubit_ham_loaded が既にある前提
def load_sparse_pauli_op(path: str):
    from qiskit.quantum_info import SparsePauliOp
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)  # [[label, [re, im]], ...]
    lst = [(label, complex(re, im)) for label, (re, im) in data]
    return SparsePauliOp.from_list(lst)


qubit_ham_loaded = load_sparse_pauli_op("qubit_ham.json")

def hermitian_real_part(op: SparsePauliOp, tol: float = 1e-12) -> SparsePauliOp:
    """H -> (H + H^\dagger)/2 でエルミート化し、係数の微小虚部を落として実数化する。"""
    # エルミート化
    Hh = (op + op.adjoint()) * 0.5
    Hh = Hh.simplify()

    # 係数の実数化（虚部が tol 以下なら消す）
    coeffs = np.array(Hh.coeffs)
    coeffs = np.where(np.abs(coeffs.imag) < tol, coeffs.real, coeffs)  # 虚部が大きいものは残す
    if np.iscomplexobj(coeffs):
        # まだ複素が残っていれば、非エルミート成分が大きいということ
        raise ValueError(
            "Hamiltonian still has significant complex coefficients after hermitianization. "
            "元の演算子が非エルミートの可能性があります。"
        )

    return SparsePauliOp(Hh.paulis, coeffs=coeffs).simplify()

qubit_ham_hr = hermitian_real_part(qubit_ham_loaded, tol=1e-12)
res = estimate_ground_energy_qpe(qubit_ham_hr, num_eval_qubits=10, shots=8192)

print("=== QPE result ===")
print(f"most likely bitstring : {res['bitstring']}")
print(f"phase φ               : {res['phase']:.8f}  (fraction of 1)")
print(f"time scale t          : {res['t']:.6e}")
print(f"estimated energy E    : {res['energy']:.12f}")


ValueError: Hamiltonian still has significant complex coefficients after hermitianization. 元の演算子が非エルミートの可能性があります。